<a href="https://colab.research.google.com/github/fernanda-palacios/ai-code-notebooks/blob/main/g_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Celebrity Facial Recognition with Transfer Learning**


## Data Partitioning

There is a daily limit on CelebA downloads through the PyTorch API. You can alternatively download the dataset by adding a shortcut to [this folder](https://drive.google.com/drive/folders/1Jvm0fVuExpzvYNQ-jwMWFQQG4C_yg4WA) in your Google Drive and running the following cell.

In [ ]:
from google.colab import drive
drive.mount("/content/drive")
!cp -r /content/drive/MyDrive/celeba /content/celeba
!unzip -q /content/celeba/img_align_celeba.zip -d /content/celeba
drive.flush_and_unmount()

Mounted at /content/drive


To ensure the validation and test sets only have images of celebrities that have been trained on and for time's sake, we will only use celebrities that have over 30 images in the dataset. To do this, we modify our partition file to reflect an 80/10/10 training/validation/test split of these celebrities and reorder the identities so we can match them to the number of output neurons.

In [ ]:
import torch
import torchvision
from math import floor, ceil
from random import seed, shuffle


seed(64)
torchvision.datasets.CelebA(
    root='./', split='all', target_type='identity', download=True)

celebs_to_images = {}
with open("/content/celeba/identity_CelebA.txt") as identity_file:
    identity_lines = identity_file.readlines()
for line in identity_lines:
    image_file, celeb = line.strip().split()
    images = celebs_to_images.setdefault(int(celeb), [])
    images.append(int(image_file[:-4]))

data_split = [3] * len(identity_lines) # 3s will be ignored by the partitioning
image_reorder = {}
reorder = 0
for celeb in celebs_to_images:
    shuffle(celebs_to_images[celeb])
    if len(celebs_to_images[celeb]) > 30:
        num_train = floor(len(celebs_to_images[celeb]) * 0.8)
        num_valid = num_train + ceil(len(celebs_to_images[celeb]) * 0.1)
        for i in range(num_train):
            data_split[celebs_to_images[celeb][i] - 1] = 0
            image_reorder[celebs_to_images[celeb][i]] = reorder
        for i in range(num_train, num_valid):
            data_split[celebs_to_images[celeb][i] - 1] = 1
            image_reorder[celebs_to_images[celeb][i]] = reorder
        for i in range(num_valid, len(celebs_to_images[celeb])):
            data_split[celebs_to_images[celeb][i] - 1] = 2
            image_reorder[celebs_to_images[celeb][i]] = reorder
        reorder += 1

print(f"There are {reorder} celebrities with over 30 images.")
for celeb in celebs_to_images:
    if len(celebs_to_images[celeb]) <= 30:
        for i in range(len(celebs_to_images[celeb])):
            image_reorder[celebs_to_images[celeb][i]] = reorder
        reorder += 1

new_lines = []
with open("/content/celeba/list_eval_partition.txt") as partition_file:
    for i, line in enumerate(partition_file):
        new_lines.append(f"{line[:-2]}{data_split[i]}\n")
with open("/content/celeba/list_eval_partition.txt", 'w') as partition_file:
    partition_file.writelines(new_lines)

new_lines = []
with open("/content/celeba/identity_CelebA.txt") as identity_file:
    for i, line in enumerate(identity_file):
        new_lines.append(f"{line[:10]} {image_reorder[i+1]}\n")
with open("/content/celeba/identity_CelebA.txt", 'w') as identity_file:
    identity_file.writelines(new_lines)

Files already downloaded and verified
There are 17 celebrities with over 30 images.


In [ ]:
torch.manual_seed(64)
class CelebA_No_Check(torchvision.datasets.CelebA):
    file_list = [] # skip file verification
celeb_train = CelebA_No_Check(
    root='./', split='train', target_type='identity',
    transform = torchvision.models.ResNet18_Weights.IMAGENET1K_V1.transforms(),
    download=False)
print(celeb_train)
print()
celeb_valid = CelebA_No_Check(
    root='./', split='valid', target_type='identity',
    transform = torchvision.models.ResNet18_Weights.IMAGENET1K_V1.transforms(),
    download=False)
print(celeb_valid)
print()
celeb_test = CelebA_No_Check(
    root='./', split='test', target_type='identity',
    transform = torchvision.models.ResNet18_Weights.IMAGENET1K_V1.transforms(),
    download=False)
print(celeb_test)
print()
celeb_datasets = {
    'train': celeb_train, 'valid': celeb_valid, 'test': celeb_test}

train_loader = torch.utils.data.DataLoader(
    celeb_train, batch_size=16, shuffle=True)
valid_loader = torch.utils.data.DataLoader(celeb_valid, batch_size=16)
test_loader = torch.utils.data.DataLoader(celeb_test, batch_size=16)
celeb_loaders = {
    'train': train_loader, 'valid': valid_loader, 'test': test_loader}

Dataset CelebA_No_Check
    Number of datapoints: 429
    Root location: ./
    Target type: ['identity']
    Split: train
    StandardTransform
Transform: ImageClassification(
               crop_size=[224]
               resize_size=[256]
               mean=[0.485, 0.456, 0.406]
               std=[0.229, 0.224, 0.225]
               interpolation=InterpolationMode.BILINEAR
           )

Dataset CelebA_No_Check
    Number of datapoints: 68
    Root location: ./
    Target type: ['identity']
    Split: valid
    StandardTransform
Transform: ImageClassification(
               crop_size=[224]
               resize_size=[256]
               mean=[0.485, 0.456, 0.406]
               std=[0.229, 0.224, 0.225]
               interpolation=InterpolationMode.BILINEAR
           )

Dataset CelebA_No_Check
    Number of datapoints: 51
    Root location: ./
    Target type: ['identity']
    Split: test
    StandardTransform
Transform: ImageClassification(
               crop_size=[224]
       

## Sample Images

In [ ]:
from torch.utils.tensorboard.writer import SummaryWriter
board = SummaryWriter()

In [ ]:
train_iter = iter(train_loader)
images, labels = next(train_iter)
board.add_images("Sample Images", images)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=runs

## Define Functions

In [ ]:
from torch import nn
import torch.nn.functional as F
from tqdm.notebook import tqdm


def process_batch(
    device, phase, dataset, loader, model, criterion, optimizer, scheduler):

    assert phase in ('train', 'valid')
    model.train() if phase == 'train' else model.eval()
    full_loss = 0.0
    n_correct = 0

    for images, labels in tqdm(loader):
        images = images.to(device)
        labels = labels.to(device)

        with torch.set_grad_enabled(phase == 'train'):
            optimizer.zero_grad()
            outputs = model(images)
            predicted = outputs.argmax(dim=1)
            loss = criterion(outputs, labels)
            if phase == 'train':
                loss.backward()
                optimizer.step()

        full_loss += loss.item() * len(labels)
        n_correct += (predicted == labels.data).sum()
    if phase == 'train' and scheduler is not None:
        scheduler.step()

    average_loss = full_loss / len(dataset)
    accuracy = n_correct / len(dataset) * 100
    return average_loss, accuracy


def train_and_validate(
    datasets, loaders, model, criterion, optimizer, scheduler=None, epochs=20):

    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print("Device:", device)
    model = model.to(device)

    for epoch in range(epochs):

        train_loss, train_accuracy = process_batch(
            device, 'train', datasets['train'], loaders['train'],
            model, criterion, optimizer, scheduler)
        print(f"Epoch {epoch+1} training average loss: {train_loss:.3f}",
              f"with {train_accuracy:.2f}% accuracy")

        valid_loss, valid_accuracy = process_batch(
            device, 'valid', datasets['valid'], loaders['valid'],
            model, criterion, optimizer, scheduler)
        print(f"Epoch {epoch+1} validation average loss: {valid_loss:.3f}",
              f"with {valid_accuracy:.2f}% accuracy")

        board.add_scalars('Average Losses',
            {"Training": train_loss, "Validation": valid_loss}, epoch)
        board.add_scalars('Accuracy',
            {"Training": train_accuracy, "Validation": valid_accuracy}, epoch)
        print()


def test(dataset, loader, model):
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    model.eval()
    n_correct = 0
    with torch.no_grad():
        for images, labels in tqdm(loader):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            predicted = outputs.argmax(dim=1)
            n_correct += (predicted == labels.data).sum()

    print(f"Test accuracy: {n_correct / len(dataset) * 100:.3f}%")

## Fixed Feature Extractor

In [ ]:
resnet_transfer = torchvision.models.resnet18(weights='IMAGENET1K_V1')
print(resnet_transfer)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 83.7MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
for weight in resnet_transfer.parameters():
    weight.requires_grad = False
resnet_transfer.fc = nn.Linear(resnet_transfer.fc.in_features, 17) # redefine last layer (we want to classify into 17 celebrities)
board.add_graph(resnet_transfer, images)

criterion = nn.CrossEntropyLoss()

transfer_optimizer = torch.optim.SGD(
    resnet_transfer.fc.parameters(), lr=0.001, momentum=0.9)

train_and_validate(
    celeb_datasets, celeb_loaders, resnet_transfer,
    criterion, transfer_optimizer)
test(celeb_test, test_loader, resnet_transfer)

Device: cuda:0


  0%|          | 0/27 [00:00<?, ?it/s]

Epoch 1 training average loss: 2.890 with 6.76% accuracy


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1 validation average loss: 2.732 with 13.24% accuracy



...

Epoch 20 training average loss: 0.591 with 93.24% accuracy


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 20 validation average loss: 1.058 with 69.12% accuracy



  0%|          | 0/4 [00:00<?, ?it/s]

Test accuracy: 74.510%


In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=runs

## Fine-Tuning

We should use the trained model from the previous section; the gradients would be too large for randomly initialized weights and risk throwing the pre-trained weights too far off.

In [ ]:
for weight in resnet_transfer.parameters():
    weight.requires_grad = True
tune_optimizer = torch.optim.SGD( # smaller learning rate to prevent overfitting
    resnet_transfer.parameters(), lr=0.0001, momentum=0.9, weight_decay=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(
    tune_optimizer, step_size=2, gamma=0.9)

train_and_validate(
    celeb_datasets, celeb_loaders, resnet_transfer,
    criterion, tune_optimizer, scheduler, 10)
test(celeb_test, test_loader, resnet_transfer)

Device: cuda:0


  0%|          | 0/27 [00:00<?, ?it/s]

Epoch 1 training average loss: 0.562 with 93.71% accuracy


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1 validation average loss: 1.009 with 72.06% accuracy



...

Epoch 10 training average loss: 0.283 with 100.00% accuracy


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 10 validation average loss: 0.867 with 80.88% accuracy



  0%|          | 0/4 [00:00<?, ?it/s]

Test accuracy: 78.431%


In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=runs

## Training from Scratch

In [ ]:
resnet_scratch = torchvision.models.resnet18() # no pre-trained weights
resnet_scratch.fc = nn.Linear(resnet_scratch.fc.in_features, 17)
scratch_optimizer = torch.optim.SGD(
    resnet_scratch.parameters(), lr=0.01, momentum=0.9)

train_and_validate(
    celeb_datasets, celeb_loaders, resnet_scratch, criterion, scratch_optimizer)
test(celeb_test, test_loader, resnet_scratch)

Device: cuda:0


  0%|          | 0/27 [00:00<?, ?it/s]

Epoch 1 training average loss: 3.058 with 7.23% accuracy


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1 validation average loss: 5.385 with 10.29% accuracy



...

Epoch 20 training average loss: 0.007 with 100.00% accuracy


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 20 validation average loss: 1.062 with 63.24% accuracy



  0%|          | 0/4 [00:00<?, ?it/s]

Test accuracy: 70.588%


In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=runs